# Random Forest Regression
Random Forest Regression is an ensemble learning method that operates by constructing a multitude of decision trees during training. For regression tasks, it outputs the mean prediction of the individual trees, which helps in reducing overfitting and improving the model's generalization.

## Advantages:
- Robustness to Overfitting: By averaging multiple trees, the model reduces the risk of overfitting.
- High Accuracy: Capable of achieving high accuracy due to the ensemble method.
- Feature Importance: Provides insights into feature importance.
- Scalability: Can handle large datasets efficiently.

## Disadvantages:
- Complexity: The model can become very complex and require significant computational resources.
- Less Interpretability: Individual trees are interpretable, but the ensemble is not.
- Slower Predictions: Making predictions can be slower compared to simpler models.

## Use Cases:
- Finance: Predicting stock prices or credit risk.
- Healthcare: Disease prediction and diagnosis.
- Marketing: Customer segmentation and sales forecasting.
- Environment: Weather forecasting and pollution level prediction.

## Scaling (not necessary)
Random Forest Regression typically does not require feature scaling. 

## Encoding (necessary)
Encoding categorical data to numerical is essential, as Random Forests can only handle numerical input.

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import uniform, loguniform
from sklearn.ensemble import RandomForestRegressor

# Read Dataset

In [2]:
df = pd.read_csv('50_StartUp_dataset.csv')
df.head()

,Unnamed: 0,R&D Spend,Administration,Marketing Spend,Profit,Florida,New York
0,0,165349.20,136897.80,471784.10,192261.83,0.0,1.0
1,1,162597.70,151377.59,443898.53,191792.06,0.0,0.0
2,2,153441.51,101145.55,407934.54,191050.39,1.0,0.0
3,3,144372.41,118671.85,383199.62,182901.99,0.0,1.0
4,4,142107.34,91391.77,366168.42,166187.94,1.0,0.0


# get X , Y

In [3]:
x=df.drop('Profit',axis=1)
y=df['Profit']

## Get train, test and valid data

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.1, random_state=42)
x_train, x_valid, y_train, y_valid=train_test_split(x_train,y_train,test_size=.1, random_state=42)

In [5]:
print('x_train shape =',x_train.shape)
print('x_test shape =',x_test.shape)
print('x_valid shape =',x_valid.shape)
print('y_train shape =',y_train.shape)
print('y_test shape =',y_test.shape)
print('y_valid shape =',y_valid.shape)

x_train shape = (40, 6)
x_test shape = (5, 6)
x_valid shape = (5, 6)
y_train shape = (40,)
y_test shape = (5,)
y_valid shape = (5,)


# Train

## Grid Search

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf_regressor = RandomForestRegressor(random_state=42)

params = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
    'bootstrap': [True, False]
}

param_grid = {
    'n_estimators': [100, 200, 300, 400, 500, 1000],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 8, 10],
    'bootstrap': [True, False],
    'criterion': ['mse', 'mae']
}


grid_search = GridSearchCV(rf_regressor, params, scoring='r2', cv=5, n_jobs=-1)

# Train the grid search
grid_search.fit(x_train, y_train)  

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4, 8],
                         'min_samples_split': [2, 5, 10, 15],
                         'n_estimators': [100, 200]},
             scoring='r2')

In [7]:
print("Best Hyperparameter Index:", grid_search.best_index_)
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validated Score:", grid_search.best_score_)

Best Hyperparameter Index: 1761
Best Hyperparameters: {'bootstrap': False, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Cross-Validated Score: 0.9502573738051112


In [25]:
# Get the model with best hyperparameters
model = grid_search.best_estimator_
y_pred = model.predict(x_test)

## Randomized Search

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf_regressor = RandomForestRegressor(random_state=42)

params = {
    'n_estimators': np.arange(50, 1001, 50),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None] + list(np.arange(10, 101, 10)),
    'min_samples_split': np.arange(2, 21, 2),
    'min_samples_leaf': np.arange(1, 21, 2),
    'bootstrap': [True, False]
}

param_dist = {
    'n_estimators': np.arange(100, 1001, 100),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None] + list(np.arange(10, 101, 10)),
    'min_samples_split': np.arange(2, 21, 2),
    'min_samples_leaf': np.arange(1, 21, 2),
    'bootstrap': [True, False],
    'criterion': ['mse', 'mae']
}

random_search = RandomizedSearchCV(rf_regressor, params, scoring='r2', n_iter=100, cv=5, n_jobs=-1, random_state=42)

# Train the random search
random_search.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30, 40, 50,
                                                      60, 70, 80, 90, 100],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20]),
                                        'n_estimators': array([  50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,
        600,  650,  700,  750,  800,  850,  900,  950, 1000])},
                   random_state=42, scoring='r2')

In [187]:
# print("Best Hyperparameter Index:", random_search.best_index_)
# print("Best Hyperparameters:", random_search.best_params_)
# print("Best Cross-Validated Score:", random_search.best_score_)

In [188]:
# model = random_search.best_estimator_
# y_pred = model.predict(x_test)

## Train RandomForestRegressor without search

In [14]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(bootstrap=False, max_depth=10, max_features='sqrt', n_estimators=200, random_state=42)
model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=10, max_features='sqrt',
                      n_estimators=200, random_state=42)

# Check overfiiting

In [15]:
y_train_pred=model.predict(x_train)
r2_score(y_train_pred , y_train)

0.9999999999093917

In [16]:
y_valid_pred=model.predict(x_valid)
r2_score(y_valid_pred , y_valid)

0.8621012070780211

# Evaluate model

In [17]:
y_pred = model.predict(x_test)

## r2_score

In [18]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
r2

0.9553326081977304

## mean_squared_error

In [19]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

30422609.253626596

## mean_absolute_error

In [20]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
mae

4118.818499999866